# Data Curation
To train a machine learning model from data, that data must first be "curated" to ensure that chemical structures 
and properties are represented consistently.
Curating raw data is a long, detailed process that takes several steps.
**[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)** strings need to be standardized, measurements
need to be converted to common units, outliers need to be removed or corrected, 
and replicates need to be combined. These steps are vital to create datasets that
can be used to train useful predictive models. 
Here we will cover some functions in **[AMPL](https://github.com/ATOMScience-org/AMPL)** that will help
you to perform these steps.

- [base_smiles_from_smiles](https://ampl.readthedocs.io/en/latest/utils.html#utils.struct_utils.base_smiles_from_smiles)
- [standardize_relations](https://ampl.readthedocs.io/en/latest/utils.html#utils.data_curation_functions.standardize_relations)
- [compute_negative_log_responses](https://ampl.readthedocs.io/en/latest/utils.html#utils.data_curation_functions.compute_negative_log_responses)
- [remove_outlier_replicates](https://ampl.readthedocs.io/en/latest/utils.html#utils.curate_data.remove_outlier_replicates)
- [aggregate_assay_data](https://ampl.readthedocs.io/en/latest/utils.html#utils.curate_data.aggregate_assay_data)

These are just a few of the steps needed to curate a dataset.

## Import Standard Data Science Packages
To use **[AMPL](https://github.com/ATOMScience-org/AMPL)**, or to do almost anything else with data, you'll need to become familiar with the popular packages 
**[pandas](https://pandas.pydata.org/)**, 
**[numpy](https://numpy.org/)**, 
**[matplotlib](https://matplotlib.org/)** and 
**[seaborn](https://seaborn.pydata.org/)**. 
When you installed **[AMPL](https://github.com/ATOMScience-org/AMPL)** you will have installed these packages as well, so you simply need to import them here.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Read the Data
We've prepared an example dataset containing **[$K_i$](https://en.wikipedia.org/wiki/Ligand_\(biochemistry\)#Receptor/ligand_binding_affinity)** values for inhibitors of 
the **[SLC6A3](https://www.ebi.ac.uk/chembl/target_report_card/CHEMBL238/)** 
dopamine transporter collected from **[ChEMBL](https://www.ebi.ac.uk/chembl/)**. This dataset is 
simpler than most that we find in the wild, but it will let us concisely 
demonstrate some **[AMPL](https://github.com/ATOMScience-org/AMPL)** curation tools. The first step of data curation is to read the raw data into a Pandas data frame.

In [2]:
# Read in data
meas="ROS"
dir="up"
raw_df = pd.read_csv(f'../datasets/training_data/ROS_hits_up_class_cur2.csv')

# raw_df=pd.read_csv('../datasets/raw_datasets/DILIRankST_smiles.csv')

In [3]:
# Check the number of rows and columns in the dataset
raw_df.shape

(802, 4)

In [4]:
# List the column names
raw_df.columns.values

array(['compound_id', 'base_rdkit_smiles', 'relation', 'active'],
      dtype=object)

## Standardize SMILES
The **[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)** grammar allows the same chemical structure to be represented by many different **[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)** strings. In addition, measurements may be performed on compounds with different salt groups or with radioisotope labels, which we treat as equivalent to the base compounds. **[AMPL](https://github.com/ATOMScience-org/AMPL)** provides a **[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)** standardization function, `base_smiles_from_smiles`, that removes salt groups and isotopes and returns a unique **[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)** string for each base compound structure. This step simplifies the machine learning problem by ensuring each compound is represented with the same set of features and multiple measurements on the same compound can be grouped together. 

> **Note:** *The input to `base_smiles_from_smiles` must be a list; numpy arrays and pandas Series objects must be converted with the tolist function.*

In [5]:
from atomsci.ddm.utils.struct_utils import base_smiles_from_smiles
# Since the base_smiles_from_smiles function can be slow, we specify the workers=8 argument
# to divide the work across 8 threads.
raw_df['base_rdkit_smiles'] = base_smiles_from_smiles(raw_df.base_rdkit_smiles.tolist(), workers=8)

/Users/echun/repos/AMPL/atomsci/ddm/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/Users/echun/repos/AMPL/atomsci/ddm/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/Users/echun/repos/AMPL/atomsci/ddm/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/Users/echun/repos/AMPL/atomsci/ddm/__init__.py:1: U

In [25]:
raw_df.loc[raw_df.compound_id=='SPID634', 'base_rdkit_smiles']=base_smiles_from_smiles(['C[C@H](CCCC(C)(C)O)[C@H]1CC[C@H]2\C(CCC[C@@]12C)=C\C=C1\C[C@@H](O)C[C@H](O)C1=C'])
raw_df['relation']='='

In [6]:
raw_df.base_rdkit_smiles.nunique()

802

For this dataset there are 1830 unique **[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)** that are standardized to 1823 unique base **[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)**. It is common for two different **[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)** strings to be standardized to the same value. From now on we will use `base_rdkit_smiles` to represent compound structures.

## Remove Outliers and Aggregate Replicate Measurements
The final step is to remove outliers and aggregate (average) replicate measurements on the same compounds. The function `remove_outlier_replicates` is a simple filter that groups measurements by compound, computes the median of each group, and removes values that differ more than `max_diff_from_median` units from the median. When the measurements are very spread out relative to `max_diff_from_median`, *all* the rows for a compound may be deleted from the dataset. The default setting ($1.0$) generally works well for $pK_i$ values.

The function `aggregate_assay_data` replaces multiple replicate measurements for each compound with a single aggregate value. Usually this is simply the average over the replicates, but if the dataset contains both censored and uncensored values for a compound, the function computes a maximum likelihood estimate that takes the censoring into account.


In [8]:
from atomsci.ddm.utils.curate_data import remove_outlier_replicates, aggregate_assay_data

curated_df = remove_outlier_replicates(raw_df, id_col='compound_id',
                                response_col='active',
                                max_diff_from_median=1.0)

curated_df = aggregate_assay_data(curated_df, 
                             value_col='active',
                             output_value_col='active',
                             id_col='compound_id',
                             smiles_col='base_rdkit_smiles',
                             relation_col='relation',
                             label_actives=False,
                             verbose=True
                        )
print("Original data shape: ", raw_df.shape)
print("Curated data shape: ", curated_df.shape)

curated_df = curated_df.drop('relation', axis=1)
curated_df.head()

/Users/echun/repos/AMPL/atomsci/ddm/utils/curate_data.py:535: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filt_df = gby.apply(filter_outliers)


0 entries in input table are missing SMILES strings
802 unique SMILES strings are reduced to 802 unique base SMILES strings
Original data shape:  (802, 4)
Curated data shape:  (802, 4)


,compound_id,base_rdkit_smiles,active
0,SPID236,C#CC1(O)CCC2C3CCC4=CC(=O)CCC4C3CCC21C,0.0
1,SPID231,C#CC1(O)CCC2C3CCC4=CC(=O)CCC4C3CCC21CC,1.0
2,SPID339,C#CCN(C)[C@H](C)Cc1ccccc1,1.0
3,FOUWCSDKDDHKQP,C#CCN1C(=O)COc2cc(F)c(N3C(=O)C4=C(CCCC4)C3=O)cc21,1.0
4,SPID475,C#CCN[C@@H]1CCc2ccccc21,0.0


In [9]:
raw_df[raw_df.base_rdkit_smiles.isin(curated_df[curated_df.active.between(0.0, 1.0, inclusive='neither')].base_rdkit_smiles)].sort_values('base_rdkit_smiles')

,compound_id,base_rdkit_smiles,relation,active


In [10]:
# check dupes
if meas=='CellCount':
    map_dict=dict(zip(curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'compound_id'],
                       [1,1,1,1,1]))
elif meas=='GSH':
    map_dict=dict(zip(curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'compound_id'],
                       [0,1,1,1,1,1]))
elif meas=='MitoStruct':
    map_dict=dict(zip(curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'compound_id'],
                       [1,1,1,1,0,1,1]))
elif meas=='MMP':
    map_dict=dict(zip(curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'compound_id'],
                       [1,1,0,1,1,0,0,1,1]))
elif meas=='NucArea':
    map_dict=dict(zip(curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'compound_id'],
                       [1,1,0,1,1,0]))
elif meas=='NucMask':
    map_dict=dict(zip(curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'compound_id'],
                       [1,1,1,1,1]))
elif meas=='ROS':
    map_dict=dict(zip(curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'compound_id'],
                       [1,1,1,1,1,1,1]))
     
curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'active']=curated_df.loc[curated_df.active.between(0.0, 1.0, inclusive='neither'), 'compound_id'].map(map_dict)


Finally, we save the curated dataset to a CSV file. 

In [11]:
curated_df.to_csv(f'../datasets/training_data/ROS_hits_up_class_cur2.csv', index=False)

# Create MT dataset

In [54]:
meas="ROS"
dir="up"
df=pd.read_csv(f'../datasets/training_data/ROS_hits_up_class_cur_with_supp.csv')
df=df.drop(columns=['relation'])
suffix1 = '_ROS'
for file in os.listdir('../datasets/training_data'):
    if 'cur' in file:
        if 'ROS' in file:
            pass
        else:
            df2=pd.read_csv(f'../datasets/training_data/{file}')
            try:
                df2=df2.drop(columns=['relation'])
            except:
                pass
            suffix2 = '_'+file.split('_')[0]
            print(file, suffix2)
            df=df.merge(df2, on=['compound_id','base_rdkit_smiles'], how='outer', suffixes=(suffix1, suffix2))
            suffix1=suffix2
df=df.rename(columns={'active':f'active{suffix2}'})
df

MMP_hits_up_class_cur.csv _MMP
CellCount_hits_dn_class_cur.csv _CellCount
MitoStruct_hits_up_class_cur.csv _MitoStruct
GSH_hits_up_class_cur.csv _GSH
NucMask_hits_up_class_cur.csv _NucMask
NucArea_hits_dn_class_cur.csv _NucArea


,compound_id,base_rdkit_smiles,active_ROS,active_MMP,active_CellCount,active_MitoStruct,active_GSH,active_NucMask,active_NucArea
0,AAEVYOVXGOFMJO,CSc1nc(NC(C)C)nc(NC(C)C)n1,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,AEMFNILZOJDQLW,C[C@]12CCC(=O)C=C1CC[C@@H]1[C@@H]2CC[C@]2(C)C(...,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,AIMMSOZBPYFASU,COc1cc(OC)nc(NC(=O)[N-]S(=O)(=O)c2ncccc2OCC(F)...,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,AKTXOQVMWSFEBQ,CC(C)(C)c1cc(/C=C2\SC(=N)NC2=O)cc(C(C)(C)C)c1O,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ALWUKGXLBSQSMA,CCCCCCC1(C)CCC(=O)O1,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
806,ZNOLGFHPUIJIMJ,COP(=S)(OC)Oc1ccc([N+](=O)[O-])c(C)c1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
807,ZOTRFGNOTDLOAU,CCC(CC)Nc1ccc(C)c(C)c1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
808,ZUBPKHVCBGWWGO,Cc1cc(SC2=C(O)C[C@@](CCc3ccc(N)cc3)(C(C)C)OC2=...,1.0,NaN,NaN,NaN,NaN,NaN,NaN
809,ZXVONLUNISGICL,Cc1cc([N+](=O)[O-])cc([N+](=O)[O-])c1O,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
df[df.base_rdkit_smiles.duplicated(keep=False)].sort_values('base_rdkit_smiles').to_csv(f'../datasets/dupes.csv', index=False)

In [56]:
df=df[~df.base_rdkit_smiles.duplicated(keep=False)]
dupes=pd.read_csv(f'../datasets/dupes_cur.csv')

df=pd.concat([df,dupes], ignore_index=True)

In [57]:
df.to_csv(f'../datasets/cell_health_MT_supps.csv', index=False)